<a href="https://colab.research.google.com/github/Ritanjali-Jena/PublicDataSet/blob/master/Fetch_Takehome.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook presents a structured approach to data analysis, divided into two main parts:
## Individual CSV File Analysis
## Combined Analysis of Joined Tables

### Part 1: Individual CSV File Analysis
### For each CSV file:
#### Load the data
#### Perform data quality checks
#### Conduct initial exploratory analysis
#### Comment on findings within each code block

### Part 2: Combined Analysis
#### Join the three CSV files
#### Perform comprehensive analysis on the combined dataset
#### Identify cross-table insights and patterns

### Instruct for users to read data files from my shared Google drive folder:
### Run all cells: You should run all the cells in the notebook for mounting the Google Drive.
### Authentication: When you reach the drive.mount cell, you will be prompted to authenticate with your own Google accounts. This is necessary to grant the Colab environment access to your Google Drives for the session.
### Grant Access: A pop-up window will appear requesting access to your Google Drives. You must allow access for the notebook to mount and read files from my shared Google Drive folders.

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install pandasql
import pandas as pd
from pandasql import sqldf

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26772 sha256=5ab62ed91921ccf0ce5a1ba9eb104c11129550a512bc044a6111046be1d44291
  Stored in directory: /root/.cache/pip/wheels/68/5d/a5/edc271b998f909801d7956959f699b976cc9896075dc47c153
Successfully built pandasql


# Load Products CSV file


In [3]:
file_path  = '/content/drive/MyDrive/Publicdata/PRODUCTS_TAKEHOME.csv'
products = pd.read_csv(file_path)
products

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE
0,Health & Wellness,Sexual Health,Conductivity Gels & Lotions,NaN,NaN,NaN,7.964944e+11
1,Snacks,Puffed Snacks,Cheese Curls & Puffs,NaN,NaN,NaN,2.327801e+10
2,Health & Wellness,Hair Care,Hair Care Accessories,NaN,PLACEHOLDER MANUFACTURER,ELECSOP,4.618178e+11
3,Health & Wellness,Oral Care,Toothpaste,NaN,COLGATE-PALMOLIVE,COLGATE,3.500047e+10
4,Health & Wellness,Medicines & Treatments,Essential Oils,NaN,MAPLE HOLISTICS AND HONEYDEW PRODUCTS INTERCHA...,MAPLE HOLISTICS,8.068109e+11
...,...,...,...,...,...,...,...
845547,Health & Wellness,Topical Muscle & Joint Relief Treatments,Braces & Wraps,NaN,NaN,NaN,7.223016e+11
845548,Snacks,Cookies,NaN,NaN,"TREEHOUSE FOODS, INC.",LOFTHOUSE,4.182082e+10
845549,Snacks,Candy,Confection Candy,NaN,HARIBO GMBH & CO KG,HARIBO,1.001672e+11
845550,Snacks,Nuts & Seeds,Hazelnuts,NaN,DOUBLE-COLA CO,JUMBO,7.539076e+10


In [4]:
from pandasql import sqldf

# Check nulls in column "Barcode"
null_query = """
SELECT barcode
FROM products
WHERE barcode IS NULL
"""

null_result = sqldf(null_query, globals())
print("Null Barcodes:")
print(null_result)

# Check duplicates in column "Barcode"
duplicate_query = """
SELECT barcode, COUNT(*) AS count
FROM products
GROUP BY barcode
HAVING COUNT(*) > 1
"""

duplicate_result = sqldf(duplicate_query, globals())
print("\nDuplicate Barcodes:")
print(duplicate_result)

Null Barcodes:
     BARCODE
0       None
1       None
2       None
3       None
4       None
...      ...
4020    None
4021    None
4022    None
4023    None
4024    None

[4025 rows x 1 columns]

Duplicate Barcodes:
          BARCODE  count
0             NaN   4025
1    4.005100e+05      2
2    4.043100e+05      2
3    4.387110e+05      2
4    5.631780e+05      2
..            ...    ...
181  8.731747e+07      2
182  8.731753e+07      2
183  3.600540e+08      2
184  1.700033e+10      2
185  5.233692e+10      2

[186 rows x 2 columns]


### We have 4025 Null records and 186 duplicate records.
### Data Quality issue on Products table: Since "Barcode" is the join key and contains null values, joining "Products" with "Transactions" on "Barcode" will result in dropped records. Additionally, any duplicate "Barcode" values will lead to a Cartesian join (cross join). Additionally relationship diagram says from Products to Transactions is 1:M however barcode has duplicates, and it violates the relation.




In [5]:
# Check how many unique category_1 we have

query = """
SELECT COUNT(DISTINCT category_1) AS unique_category1_count
FROM products
"""

result = sqldf(query, globals())
print(result)

   unique_category1_count
0                      27


In [6]:
# Print all category_1
query = """
SELECT DISTINCT category_1
FROM products
ORDER BY category_1
"""

result = sqldf(query, globals())
print(result)

                CATEGORY_1
0                     None
1                  Alcohol
2   Animals & Pet Supplies
3    Apparel & Accessories
4     Arts & Entertainment
5           Baby & Toddler
6                   Beauty
7                Beverages
8                    Dairy
9            Deli & Bakery
10             Electronics
11                  Frozen
12       Health & Wellness
13           Home & Garden
14      Household Supplies
15          Luggage & Bags
16                  Mature
17          Meat & Seafood
18                   Media
19            Needs Review
20         Office & School
21                  Pantry
22                 Produce
23              Restaurant
24                  Snacks
25          Sporting Goods
26            Toys & Games
27        Vehicles & Parts


In [7]:
# Check if we can make barcode, manufacturer and brand as composite primary key
query = """
SELECT
    CASE
        WHEN COUNT(*) = COUNT(DISTINCT barcode || manufacturer || brand)
        THEN 'barcode, manufacturer, and brand can be a primary key'
        ELSE 'barcode, manufacturer, and brand cannot be a primary key'
    END AS primary_key_check
FROM products;

"""

result = sqldf(query, globals())
print(result)

                                   primary_key_check
0  barcode, manufacturer, and brand cannot be a p...


In [8]:
# Check if each record is unique across table, so all columns combinely can be used as compoposite primary key

query = """
SELECT
    CASE
        WHEN COUNT(*) = COUNT(DISTINCT category_1 || category_2 || category_3 || category_4 || barcode || manufacturer || brand)
        THEN 'All combinations are unique'
        ELSE 'There are duplicate combinations'
    END AS uniqueness_check
FROM products
"""

result = sqldf(query, globals())
print(result)

                   uniqueness_check
0  There are duplicate combinations


### Data Quality issue on Products table: We don't have primary key in products table

In [9]:
# Test Script to find which records are duplicate (with same barcode)
query = """
SELECT
cast(barcode as number) || category_1 || category_2 || category_3 || category_4 || manufacturer || brand AS unique_combination
, count(*) AS count
FROM products
GROUP BY unique_combination
HAVING count > 1
"""

result = sqldf(query, globals())
print(result)

                                  unique_combination   count
0                                               None  793651
1  4138891.0Health & WellnessSkin CareLip Balms &...       2
2  4904403.0BeveragesCarbonated Soft DrinksColaRe...       2


In [10]:
query = """
select * from products
where barcode like '4138891%'
"""

result = sqldf(query, globals())
print(result)

          CATEGORY_1 CATEGORY_2              CATEGORY_3  \
0  Health & Wellness  Skin Care  Lip Balms & Treatments   
1  Health & Wellness  Skin Care  Lip Balms & Treatments   

                 CATEGORY_4 MANUFACTURER    BRAND    BARCODE  
0  Medicated Lip Treatments  BLISTEX INC  BLISTEX  4138891.0  
1  Medicated Lip Treatments  BLISTEX INC  BLISTEX  4138891.0  


# Products table findings
## Issue1: No primary key
## Issue2: It will either drop or do cross join due to duplicate and null values on join key (barcode)
## Issue3: Column names like Category 1, 2, 3, and 4 could have been renamed better. Assuming it's a product hierarchy, it's not clear what the hierarchy relation is for example, if Category 1 is the parent of Categories 2, 3, and 4 or if there's a different order.

# Load Transactions table

In [12]:
file_path = '/content/drive/MyDrive/Publicdata/TRANSACTION_TAKEHOME.csv'
transactions = pd.read_csv(file_path)
transactions

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
0,0000d256-4041-4a3e-adc4-5623fb6e0c99,2024-08-21,2024-08-21 14:19:06.539 Z,WALMART,63b73a7f3d310dceeabd4758,1.530001e+10,1.00,
1,0001455d-7a92-4a7b-a1d2-c747af1c8fd3,2024-07-20,2024-07-20 09:50:24.206 Z,ALDI,62c08877baa38d1a1f6c211a,NaN,zero,1.49
2,00017e0a-7851-42fb-bfab-0baa96e23586,2024-08-18,2024-08-19 15:38:56.813 Z,WALMART,60842f207ac8b7729e472020,7.874223e+10,1.00,
3,000239aa-3478-453d-801e-66a82e39c8af,2024-06-18,2024-06-19 11:03:37.468 Z,FOOD LION,63fcd7cea4f8442c3386b589,7.833997e+11,zero,3.49
4,00026b4c-dfe8-49dd-b026-4c2f0fd5c6a1,2024-07-04,2024-07-05 15:56:43.549 Z,RANDALLS,6193231ae9b3d75037b0f928,4.790050e+10,1.00,
...,...,...,...,...,...,...,...,...
49995,b5cd61a9-8033-4913-a5c4-fb3f65e3a321,2024-08-21,2024-08-31 14:13:08.634 Z,TARGET,6154bcf098f885648de2f299,8.523911e+10,2.00,1.18
49996,e1b2f634-c9ad-4152-b662-4b22efc25862,2024-08-11,2024-08-11 18:15:56.736 Z,STOP & SHOP,60aa809f188b926b2244c974,4.610040e+10,1.00,2.00
49997,b07ef8dd-e444-40a2-819b-f74a3e5f1ae7,2024-07-11,2024-07-11 08:03:25.816 Z,WALMART,60bd26e83dc3b13a15c5f4e7,6.466300e+11,1.00,20.96
49998,42475141-bef4-4df2-aa37-72577e2512bb,2024-06-18,2024-06-18 19:57:32.211 Z,MARKET BASKET,6169912fac47744405af62b7,4.180050e+10,1.00,3.00


In [13]:
# Check null values in user_id and barcode
query = """
SELECT
  COUNT(CASE WHEN user_id IS NULL THEN 1 END) AS null_user_id_count,
  COUNT(CASE WHEN barcode IS NULL THEN 1 END) AS null_barcode_count
FROM transactions;
"""

result = sqldf(query, globals())
print(result)

   null_user_id_count  null_barcode_count
0                   0                5762


### Check if receipt id is unique or not

In [14]:
query = """
SELECT receipt_id, count(*)
from transactions
group by receipt_id
having count(*)> 1
"""

result = sqldf(query, globals())
print(result)

                                 RECEIPT_ID  count(*)
0      0000d256-4041-4a3e-adc4-5623fb6e0c99         2
1      0001455d-7a92-4a7b-a1d2-c747af1c8fd3         2
2      00017e0a-7851-42fb-bfab-0baa96e23586         2
3      000239aa-3478-453d-801e-66a82e39c8af         2
4      00026b4c-dfe8-49dd-b026-4c2f0fd5c6a1         2
...                                     ...       ...
24435  fff60bb7-bf93-4e6b-8055-4d7688da7ead         2
24436  fff6c6c8-e002-4262-85ea-25849d9721db         2
24437  fffbb112-3cc5-47c2-b014-08db2f87e0c7         2
24438  fffbfb2a-7c1f-41c9-a5da-628fa7fcc746         2
24439  fffe8012-7dcf-4d84-b6c6-feaacab5074a         2

[24440 rows x 2 columns]


### Does purchase date and store name has null

In [15]:
query = """
SELECT
  COUNT(CASE WHEN purchase_date IS NULL THEN 1 END) AS null_purchase_date_count,
  COUNT(CASE WHEN store_name IS NULL THEN 1 END) AS null_store_name_count
FROM transactions;
"""

result = sqldf(query, globals())
result

,null_purchase_date_count,null_store_name_count
0,0,0


### Does each records has sale amounts

In [16]:
query = """
SELECT
  COUNT(CASE WHEN final_sale IS NULL THEN 1 END) AS null_sales_amount_count
FROM transactions;
"""

result = sqldf(query, globals())
result

,null_sales_amount_count
0,0


# Transactions table findings
### We have 5762 records with no barcode it implies that 5762 transactions has no products details.
### One good aspect is that at least each transaction is tied to a user, so there are no orphan transaction records
### Scan_date: I assume its the date when users scan their receipts, and purchase date is the date when transactions happened.

# Load user table

In [17]:
file_path = '/content/drive/MyDrive/Publicdata/USER_TAKEHOME.csv'
users = pd.read_csv(file_path)
users

,ID,CREATED_DATE,BIRTH_DATE,STATE,LANGUAGE,GENDER
0,5ef3b4f17053ab141787697d,2020-06-24 20:17:54.000 Z,2000-08-11 00:00:00.000 Z,CA,es-419,female
1,5ff220d383fcfc12622b96bc,2021-01-03 19:53:55.000 Z,2001-09-24 04:00:00.000 Z,PA,en,female
2,6477950aa55bb77a0e27ee10,2023-05-31 18:42:18.000 Z,1994-10-28 00:00:00.000 Z,FL,es-419,female
3,658a306e99b40f103b63ccf8,2023-12-26 01:46:22.000 Z,NaN,NC,en,NaN
4,653cf5d6a225ea102b7ecdc2,2023-10-28 11:51:50.000 Z,1972-03-19 00:00:00.000 Z,PA,en,female
...,...,...,...,...,...,...
99995,61fc06d41febf771966da8fa,2022-02-03 16:46:12.000 Z,1992-03-16 08:00:00.000 Z,CA,en,female
99996,6391e7ef90ad5449ec5f782d,2022-12-08 13:34:39.000 Z,1993-09-23 05:00:00.000 Z,MO,en,female
99997,637d5efdd6f2a49c49934dcb,2022-11-22 23:45:05.000 Z,1983-04-19 00:00:00.000 Z,RI,en,female
99998,5f0de23b05d8a6147dc0cafa,2020-07-14 16:50:04.000 Z,1995-06-09 04:00:00.000 Z,DE,en,female


In [18]:
# Check if id column has unique value and no null values
query = """
SELECT
    CASE
        WHEN COUNT(DISTINCT id) = COUNT(*) AND COUNT(CASE WHEN id IS NULL THEN 1 END) = 0
        THEN 'id is primary key'
        ELSE 'id is not primary key'
    END AS id_check
FROM users;
"""

result = sqldf(query, globals())
print(result)

            id_check
0  id is primary key


### Check if account was created before the user was born

In [19]:
query = """
SELECT COUNT(*)
FROM users
WHERE birth_date > created_date;
"""

result = sqldf(query, globals())
result

,COUNT(*)
0,1


In [20]:
query = """
SELECT * FROM users
WHERE birth_date > created_date;
"""

result = sqldf(query, globals())
result

,ID,CREATED_DATE,BIRTH_DATE,STATE,LANGUAGE,GENDER
0,5f31fc048fa1e914d38d6952,2020-08-11 02:01:41.000 Z,2020-10-02 15:27:28.000 Z,CA,None,None


# Users table findings
### There is one record where the account was created before the user was born. This is definitely a data quality issue. We need to implement a date check constraint on the created_date field, preventing the acceptance of any record where the user's birth date is later than the account creation date.

# Join all 3 tables and analyse data quality

### Check if purchase happened before user created_date

In [21]:
query = """
SELECT
    t.purchase_date,
    u.created_date
FROM users u
JOIN transactions t ON u.id = t.user_id
JOIN products p ON p.barcode = t.barcode
WHERE t.purchase_date < u.created_date;
"""

result = sqldf(query, globals())
print(result)

Empty DataFrame
Columns: [PURCHASE_DATE, CREATED_DATE]
Index: []


### Check if different manufacturer has same brand

In [22]:
query = """
SELECT brand,
GROUP_CONCAT(DISTINCT manufacturer) AS manufacturer,
COUNT(DISTINCT manufacturer) AS manufacturer_count
FROM products
GROUP BY brand
HAVING COUNT(DISTINCT manufacturer) > 1;
"""

result = sqldf(query, globals())
print(result)

                  BRAND                                       manufacturer  \
0             CHAPSTICK                             HALEON,GLAXOSMITHKLINE   
1  LE PETIT MARSEILIAIS  JOHNSON & JOHNSON,J AND J CONSUMER PRODUCTS IN...   
2                 TYGAZ                                      TYGAZ,UNKNOWN   

   manufacturer_count  
0                   2  
1                   3  
2                   2  


### Here we can see one brand is manufactured by multiple manufacturer example CHAPSTICK manufactured by 2 different company HALEON,GLAXOSMITHKLINE. We need more clairity why CHAPSTICK, LE PETIT MARSEILIAIS has more than 1 manufacturer, are they sister companies or its a data quality issue.
### Data Issue: product name are missing, it is better to have a product name along with the barcode.

## Is there any user_id and barcode in transactions table not in users and products table ?

In [23]:
query = """
SELECT *
FROM transactions t
WHERE t.user_id NOT IN (SELECT id FROM users)
"""
result = sqldf(query, globals())
result

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
0,0000d256-4041-4a3e-adc4-5623fb6e0c99,2024-08-21,2024-08-21 14:19:06.539 Z,WALMART,63b73a7f3d310dceeabd4758,1.530001e+10,1.00,
1,0001455d-7a92-4a7b-a1d2-c747af1c8fd3,2024-07-20,2024-07-20 09:50:24.206 Z,ALDI,62c08877baa38d1a1f6c211a,NaN,zero,1.49
2,00017e0a-7851-42fb-bfab-0baa96e23586,2024-08-18,2024-08-19 15:38:56.813 Z,WALMART,60842f207ac8b7729e472020,7.874223e+10,1.00,
3,000239aa-3478-453d-801e-66a82e39c8af,2024-06-18,2024-06-19 11:03:37.468 Z,FOOD LION,63fcd7cea4f8442c3386b589,7.833997e+11,zero,3.49
4,00026b4c-dfe8-49dd-b026-4c2f0fd5c6a1,2024-07-04,2024-07-05 15:56:43.549 Z,RANDALLS,6193231ae9b3d75037b0f928,4.790050e+10,1.00,
...,...,...,...,...,...,...,...,...
49733,b5cd61a9-8033-4913-a5c4-fb3f65e3a321,2024-08-21,2024-08-31 14:13:08.634 Z,TARGET,6154bcf098f885648de2f299,8.523911e+10,2.00,1.18
49734,e1b2f634-c9ad-4152-b662-4b22efc25862,2024-08-11,2024-08-11 18:15:56.736 Z,STOP & SHOP,60aa809f188b926b2244c974,4.610040e+10,1.00,2.00
49735,b07ef8dd-e444-40a2-819b-f74a3e5f1ae7,2024-07-11,2024-07-11 08:03:25.816 Z,WALMART,60bd26e83dc3b13a15c5f4e7,6.466300e+11,1.00,20.96
49736,42475141-bef4-4df2-aa37-72577e2512bb,2024-06-18,2024-06-18 19:57:32.211 Z,MARKET BASKET,6169912fac47744405af62b7,4.180050e+10,1.00,3.00


### Who are these users in transactions table they are not available in users table ?

In [24]:
query = """
SELECT *
FROM transactions t
WHERE t.barcode NOT IN (SELECT barcode FROM products);
"""
result = sqldf(query, globals())
result

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE


### Is there any transactions where quantity = 0 but sale <> 0

In [25]:
query = """
SELECT *
FROM transactions
WHERE final_quantity in ('zero', 0, null) AND final_sale <> 0;
"""
result = sqldf(query, globals())
result

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
0,0001455d-7a92-4a7b-a1d2-c747af1c8fd3,2024-07-20,2024-07-20 09:50:24.206 Z,ALDI,62c08877baa38d1a1f6c211a,NaN,zero,1.49
1,000239aa-3478-453d-801e-66a82e39c8af,2024-06-18,2024-06-19 11:03:37.468 Z,FOOD LION,63fcd7cea4f8442c3386b589,7.833997e+11,zero,3.49
2,0002d8cd-1701-4cdd-a524-b70402e2dbc0,2024-06-24,2024-06-24 19:44:54.247 Z,WALMART,5dcc6c510040a012b8e76924,6.811314e+11,zero,1.46
3,00096c49-8b04-42f9-88ce-941c5e06c4a7,2024-08-19,2024-08-21 17:35:21.902 Z,TARGET,6144f4f1f3ef696919f54b5c,7.830007e+10,zero,3.59
4,0010d87d-1ad2-4e5e-9a25-cec736919d15,2024-08-04,2024-08-04 18:01:47.787 Z,ALDI,66686fc2e04f743a096ea808,NaN,zero,2.29
...,...,...,...,...,...,...,...,...
12495,ffebda11-ff18-46e0-8455-cef0679558cc,2024-08-03,2024-08-13 18:36:52.294 Z,DOLLAR GENERAL STORE,65935f6499b40f103b64fc0c,7.940059e+10,zero,4.25
12496,ffef10cb-52ac-4123-aa87-0d65e85b6ef7,2024-07-21,2024-07-27 19:44:19.898 Z,WALMART,65bfb83d7050d0a6206c7b15,7.940045e+10,zero,6.97
12497,fff60bb7-bf93-4e6b-8055-4d7688da7ead,2024-08-29,2024-08-29 23:47:50.471 Z,WALMART,5eb887f71699c213b1abe2e1,7.874221e+10,zero,1.23
12498,fffbb112-3cc5-47c2-b014-08db2f87e0c7,2024-07-30,2024-08-04 11:43:31.474 Z,WALMART,5eb59d6be7012d13941af5e2,8.180000e+11,zero,4.88


### The Final_Quantity column data type could have been an integer/float type; string/varchar does not make much sense. Additionally, why is there a sale amount when the user does not buy anything? We need more clarity on this.

# Closed-ended questions:

### Since product table barcode has duplicate records its really hard to identify to select which record to do join with transactions, to answer below questions I am taking distinct barcode, and brand from products where barcode is not null and brand is not null.

In [26]:
query = """
SELECT cast(barcode as number), brand, COUNT(*) AS count
FROM products
GROUP BY barcode, brand
having  COUNT(*) > 1"""
result = sqldf(query, globals())
print(result)

     cast(barcode as number)         BRAND  count
0                        NaN          None    247
1                        NaN     100 GRAND      3
2                        NaN  3 MUSKETEERS      3
3                        NaN    4505 MEATS      3
4                        NaN         5 GUM      3
..                       ...           ...    ...
925               87108538.0        MENTOS      2
926               87304787.0        MENTOS      2
927               87306286.0        MENTOS      2
928               87317466.0        MENTOS      2
929               87317527.0        MENTOS      2

[930 rows x 3 columns]


In [27]:
query = """
SELECT *
FROM products
where cast(barcode as number) = '87108538.0' and brand = 'MENTOS'
"""
result = sqldf(query, globals())
print(result)


  CATEGORY_1 CATEGORY_2        CATEGORY_3 CATEGORY_4        MANUFACTURER  \
0     Snacks      Candy             Mints       None  PERFETTI VAN MELLE   
1     Snacks      Candy  Confection Candy       None  PERFETTI VAN MELLE   

    BRAND     BARCODE  
0  MENTOS  87108538.0  
1  MENTOS  87108538.0  


In [28]:
query = """
SELECT cast(barcode as number), brand, COUNT(distinct barcode || brand) AS distinct_count
FROM products
GROUP BY barcode, brand
having distinct_count > 1"""
result = sqldf(query, globals())
print(result)

Empty DataFrame
Columns: [cast(barcode as number), BRAND, distinct_count]
Index: []


### Above analysis shows if we take distinct barcode and barcode then we do not have duplicate

## What are the top 5 brands by receipts scanned among users 21 and over?

In [29]:
query = """
SELECT p.brand, COUNT(DISTINCT t.receipt_id) AS receipt_count
FROM transactions t
JOIN users u ON t.user_id = u.id
JOIN (select distinct barcode, brand from products where barcode is not null and brand is not null) p ON t.barcode = p.barcode
WHERE u.birth_date <= DATE('now', '-21 years')
GROUP BY p.brand
ORDER BY receipt_count DESC
LIMIT 5;
"""

result = sqldf(query, globals())
print(result)

             brand  receipt_count
0      NERDS CANDY              3
1             DOVE              3
2          TRIDENT              2
3  SOUR PATCH KIDS              2
4           MEIJER              2


## What are the top 5 brands by sales among users that have had their account for at least six months?


In [30]:
query = """
SELECT p.brand, SUM(t.final_sale) AS total_sales
FROM transactions t
JOIN users u ON t.user_id = u.id
JOIN(select distinct barcode, brand from products where barcode is not null and brand is not null) p ON t.barcode = p.barcode
WHERE u.created_date <= DATE('now', '-6 months')
GROUP BY p.brand
ORDER BY total_sales DESC
LIMIT 5;
"""

result = sqldf(query, globals())
print(result)

         brand  total_sales
0          CVS        72.00
1      TRIDENT        46.72
2         DOVE        42.88
3  COORS LIGHT        34.96
4       QUAKER        16.60


In [31]:
query =""" select distinct barcode, brand from products where barcode is not null """

result = sqldf(query, globals())
print(result)

             BARCODE            BRAND
0       7.964944e+11             None
1       2.327801e+10             None
2       4.618178e+11          ELECSOP
3       3.500047e+10          COLGATE
4       8.068109e+11  MAPLE HOLISTICS
...              ...              ...
841353  7.223016e+11             None
841354  4.182082e+10        LOFTHOUSE
841355  1.001672e+11           HARIBO
841356  7.539076e+10            JUMBO
841357  7.967933e+11          NEXCARE

[841358 rows x 2 columns]


In [32]:
query = """ select DATE('now', '-6 months') """
result = sqldf(query, globals())
print(result)

  DATE('now', '-6 months')
0               2024-07-28


## What is the percentage of sales in the Health & Wellness category by generation?

In [33]:
query = """ WITH UserGeneration AS (
    SELECT
        id,
        CASE
            WHEN birth_date BETWEEN DATE('now', '-10 years') AND DATE('now') THEN 'Gen Alpha'
            WHEN birth_date BETWEEN DATE('now', '-25 years') AND DATE('now', '-11 years') THEN 'Gen Z'
            WHEN birth_date BETWEEN DATE('now', '-40 years') AND DATE('now', '-26 years') THEN 'Millennials'
            WHEN birth_date BETWEEN DATE('now', '-56 years') AND DATE('now', '-41 years') THEN 'Gen X'
            WHEN birth_date BETWEEN DATE('now', '-76 years') AND DATE('now', '-57 years') THEN 'Baby Boomers'
            ELSE 'Silent Generation'
        END AS generation
    FROM users
),
HealthWellnessSales AS (
    SELECT
        ug.generation,
        SUM(t.final_sale) AS total_sales
    FROM transactions t
    JOIN (select distinct barcode, brand from products where barcode is not null and brand is not null and
    category_1 = 'Health & Wellness') p ON t.barcode = p.barcode
    JOIN UserGeneration ug ON t.user_id = ug.id
    GROUP BY ug.generation
),
TotalSales AS (
    SELECT SUM(final_sale) AS total_all_sales FROM transactions
)
SELECT
    hws.generation,
    hws.total_sales,
    ts.total_all_sales,
    round((hws.total_sales * 100.0 / ts.total_all_sales), 2) AS percentage_sales
FROM HealthWellnessSales hws
CROSS JOIN TotalSales ts
ORDER BY hws.generation; """


result = sqldf(query, globals())
print(result)

          generation  total_sales  total_all_sales  percentage_sales
0       Baby Boomers        92.88         171614.4              0.05
1              Gen X        58.63         171614.4              0.03
2        Millennials        36.18         171614.4              0.02
3  Silent Generation         1.97         171614.4              0.00


# Open-ended questions:
## Who are Fetch’s power users?
### Assuming max scans are consider as power user.

In [34]:
query = """ WITH UserReceiptCounts AS (
    SELECT
        user_id,
        COUNT(DISTINCT receipt_id) AS receipt_count -- Count distinct receipts for each user
    FROM transactions
    GROUP BY user_id
),
RankedUsers AS (
    SELECT
        user_id,
        receipt_count,
        DENSE_RANK() OVER (ORDER BY receipt_count DESC) AS user_rank -- Rank users by receipt count
    FROM UserReceiptCounts
)
SELECT
    user_id,
    receipt_count -- Select user_id and receipt_count
FROM RankedUsers
WHERE user_rank = 1; -- Filter for users with rank 1 (highest receipt count) """

result = sqldf(query, globals())
print(result)

                    user_id  receipt_count
0  62925c1be942f00613f7365e             10
1  64e62de5ca929250373e6cf5             10


### Since we do not have any user name or any details, so we should do it by generation to make it more meaningful

In [35]:
query = """ WITH UserGeneration AS (
    SELECT
        id,
        CASE
            WHEN birth_date BETWEEN DATE('now', '-10 years') AND DATE('now') THEN 'Gen Alpha'
            WHEN birth_date BETWEEN DATE('now', '-25 years') AND DATE('now', '-11 years') THEN 'Gen Z'
            WHEN birth_date BETWEEN DATE('now', '-40 years') AND DATE('now', '-26 years') THEN 'Millennials'
            WHEN birth_date BETWEEN DATE('now', '-56 years') AND DATE('now', '-41 years') THEN 'Gen X'
            WHEN birth_date BETWEEN DATE('now', '-76 years') AND DATE('now', '-57 years') THEN 'Baby Boomers'
            ELSE 'Silent Generation'
        END AS generation
    FROM users
),
UserReceiptCounts AS (
    SELECT
        ug.generation,
        COUNT(DISTINCT t.receipt_id) AS receipt_count
    FROM transactions t
    JOIN UserGeneration ug ON t.user_id = ug.id
    GROUP BY ug.generation
),
RankedUsers AS (
    SELECT
        generation,
        receipt_count,
        dense_rank() OVER (PARTITION BY generation ORDER BY receipt_count DESC) AS rank_within_generation
    FROM UserReceiptCounts
)
SELECT
    generation,
    receipt_count
FROM RankedUsers
WHERE rank_within_generation = 1
order by receipt_count desc """

result = sqldf(query, globals())
print(result)

          generation  receipt_count
0              Gen X             40
1       Baby Boomers             39
2        Millennials             29
3  Silent Generation             17


## The given data says Gen x are power user. However, knowing fetch business and the data from Fetch website says Nearly one-third of the app's users are under 25. So the power users are Gen Z. Source link: https://business.fetch.com/trends-and-insights/fetch-and-gen-z


## Which is the leading brand in the Dips & Salsa category?


In [36]:
query = """
SELECT p.brand, SUM(t.final_sale) AS total_sales
FROM transactions t
JOIN (select distinct barcode, brand from products where barcode is not null and brand is not null
and category_2 = 'Dips & Salsa') p ON t.barcode = p.barcode
GROUP BY p.brand
ORDER BY total_sales DESC
LIMIT 1;
"""

result = sqldf(query, globals())
print(result)

      brand  total_sales
0  TOSTITOS       260.99


### Data says TOSTITOS  is leading brand so does the external source Link : https://fetch.com/blog/smart-shopping/the-best-chips-and-dip-combos-according-to-science-fetch-has-the-receipts

## At what percent has Fetch grown year over year?
### Ans: Based on publicly available information, Fetch Rewards reportedly experienced a year-over-year growth of 208% around 2021. To gain a deeper understanding of their business model, revenue streams, and growth calculations, further engagement with Fetch representatives would be valuable. Looking forward to connect with you.